In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('student35.csv')
#df = df[df['Train/Valid/Test'] != 'Test']

In [2]:
df.can_smiles_solvent.unique()

array(['O', 'CCCCCC', 'CCCCCCCC', ..., 'CC(C)CC(=O)CC(C)C', 'C#CCCCCCC',
       'CCCCC(C)C(C)C'], dtype=object)

In [3]:
# cf. Number of data points for each fold for the reference for k-fold after groupby solvents/solutes.

''''
1048 82893
1048 82888
1048 82886
1047 82882
1047 82877
1047 82874
1047 82871
1047 82868
1047 82865
1047 82861
'''

"'\n1048 82893\n1048 82888\n1048 82886\n1047 82882\n1047 82877\n1047 82874\n1047 82871\n1047 82868\n1047 82865\n1047 82861\n"

In [4]:
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm

In [8]:
#groupby solvents

#####
solvent_smiles = []
sub_df_lengths = []
for solvent, sub_df in df.groupby('can_smiles_solvent'):
    solvent_smiles.append(solvent)
    sub_df_lengths.append(len(sub_df))
    
sub_df_lengths = np.array(sub_df_lengths)
    
lengths_argsort = np.flip(np.argsort(sub_df_lengths))

sub_df_lengths = sub_df_lengths[lengths_argsort]
solvent_smiles = [solvent_smiles[x] for x in lengths_argsort]
#####


In [9]:
print([x for x in sub_df_lengths])

[4417, 4071, 4008, 4003, 3983, 3947, 3941, 3940, 3917, 3911, 3908, 3907, 3906, 3894, 3886, 3885, 3885, 3885, 3878, 3876, 3873, 3870, 3867, 3864, 3863, 3861, 3856, 3854, 3854, 3852, 3852, 3849, 3849, 3843, 3843, 3842, 3837, 3832, 3831, 3830, 3829, 3829, 3826, 3824, 3824, 3820, 3819, 3819, 3818, 3817, 3815, 3815, 3813, 3813, 3813, 3811, 3809, 3808, 3807, 3807, 3806, 3803, 3802, 3802, 3800, 3793, 3791, 3790, 3788, 3787, 3787, 3784, 3780, 3779, 3776, 3774, 3774, 3772, 3772, 3770, 3770, 3769, 3769, 3766, 3765, 3763, 3761, 3759, 3758, 3757, 3755, 3755, 3754, 3751, 3745, 3745, 3743, 3739, 3736, 3731, 3728, 3728, 3727, 3727, 3727, 3727, 3727, 3725, 3719, 3719, 3718, 3718, 3717, 3714, 3708, 3698, 3686, 3679, 3673, 3673, 3672, 3671, 3666, 3654, 3652, 3650, 3642, 3631, 3621, 3615, 3605, 3600, 3595, 3592, 3577, 3574, 3546, 3541, 3528, 3479, 3468, 3413, 3380, 3375, 3350, 3323, 3313, 3305, 3292, 3281, 3279, 3239, 3232, 3216, 3207, 3206, 3204, 3201, 3188, 3184, 3181, 3175, 3169, 3168, 3165, 3163, 316

In [13]:
from sklearn.model_selection import KFold
DFs_eachfold = []
for i in range(11):
    new_df = pd.DataFrame(columns = df.columns)
    DFs_eachfold.append(new_df)


def build_df_eachfold(DFs_eachfold, indices_for_fold_split, random_state):
    kfold = KFold(n_splits = 11, shuffle = True, random_state = random_state)
    
    if len(indices_for_fold_split) < 11:
        fold_indices_to_consider = np.random.choice(11, len(indices_for_fold_split))
        
        for j, fold_num in enumerate(fold_indices_to_consider):
            solvent_smiles_this_fold = [solvent_smiles[indices_for_fold_split[j]]]
            DFs_eachfold[fold_num] = pd.concat([DFs_eachfold[fold_num],
                                                df[df.can_smiles_solvent.isin(solvent_smiles_this_fold)] ])            
    else:
        for fold_num in range(11):
            solvent_indices_this_fold = list(kfold.split(indices_for_fold_split))[fold_num][1]
            
            solvent_indices_this_fold = [x + min(indices_for_fold_split) for x in solvent_indices_this_fold]
            #print(solvent_indices_this_fold)
            
            solvent_smiles_this_fold = [solvent_smiles[x] for x in solvent_indices_this_fold]

            DFs_eachfold[fold_num] = pd.concat([DFs_eachfold[fold_num],
                                                df[df.can_smiles_solvent.isin(solvent_smiles_this_fold)] ])
        
    return DFs_eachfold


random_state = 0

for i in tqdm(range(0,len(sub_df_lengths), 11)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, list(range(i, min(i+11, len(sub_df_lengths)))), random_state)

'''
DFs_eachfold = build_df_eachfold(DFs_eachfold, [ i for i, x in enumerate(sub_df_lengths) if x >= 184], random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, [ i for i, x in enumerate(sub_df_lengths) if x >= 149 and x < 184], 
                                 random_state)


DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 138 and x < 149], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 130 and x < 138], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 126 and x < 130], 
                                 random_state)

for j in tqdm(range(125,38,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state)    

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 33 and x < 39], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x == 32], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 28 and x < 32], 
                                 random_state)

for j in tqdm(range(27,22,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state)  

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 14 and x < 23], 
                                 random_state)

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 12 and x < 14]  , 
                                 random_state)    

DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                 [ i for i, x in enumerate(sub_df_lengths) if x >= 3 and x < 12] , 
                                 random_state)

for j in tqdm(range(2,0,-1)):
    DFs_eachfold = build_df_eachfold(DFs_eachfold, 
                                     [ i for i, x in enumerate(sub_df_lengths) if x == j], 
                                     random_state) 
'''

for fold_num in range(11):
    print(len(DFs_eachfold[fold_num]))


100%|██████████| 132/132 [00:38<00:00,  3.41it/s]

85174
83167
85567
82701
84748
86277
84130
83453
85442
86850
85000


In [14]:
for fold_num in range(11):
    curr_df = DFs_eachfold[fold_num]
    print(
            len(curr_df[curr_df['Exp_or_QM'] == 'Exp']), len(curr_df[curr_df['Exp_or_QM'] == 'QM'])
         )
    
    curr_df['fold_number'] = fold_num

706 84468
1127 82040
1101 84466
1131 81570
979 83769
1065 85212
648 83482
1043 82410
770 84672
2079 84771
988 84012


In [15]:
pd.concat(DFs_eachfold).to_csv('student35_kfold_groupby_solvents.csv', index = False)

In [16]:
df = pd.read_csv('student35_kfold_groupby_solvents.csv')
df

,can_smiles_solute,can_smiles_solvent,DGsolv,index,Exp_or_QM,Train/Valid/Test,predicted,fold_number
0,Cc1cnccn1,CCCCCCCC,-4.700,15735,Exp,Train,-4.963775,0
1,Cc1ccc(O)c(O)c1,CCCCCCCC,-7.020,15033,Exp,Train,-6.746832,0
2,CCCOC(C)=O,CCCCCCCC,-4.055,10067,Exp,Train,-4.039470,0
3,CCCCCC,CCCCCCCC,-3.920,7752,Exp,Train,-3.797641,0
4,CCC(=O)OC,CCCCCCCC,-3.570,5761,Exp,Train,-3.480342,0
...,...,...,...,...,...,...,...,...
932504,CCCCCc1ccccc1,CCCCCc1ccccc1,-7.520,9353,Exp,Train,-7.431030,10
932505,CCCCOC(C)C,CCCCOC(C)C,-5.000,9683,Exp,Train,-4.816920,10
932506,CCNc1ccccc1,CCNc1ccccc1,-7.990,10644,Exp,Train,-7.978447,10
932507,CCOC(=O)CC(C)C,CCOC(=O)CC(C)C,-5.710,10899,Exp,Valid,-5.777698,10


In [17]:
print(len(df[(df['Exp_or_QM'] == 'Exp') & (df['fold_number'] == 10)]))

988


In [ ]:
print(len(df[ (df['fold_number'] == 10)]))

In [19]:
df[(df['Exp_or_QM'] == 'Exp') & (df['fold_number'] == 10)].can_smiles_solvent.unique()

array(['C=CCCCCCCCCCCCCCC', 'CCCCCCCCBr', 'CCC#N',
       'Cc1ccccc1[N+](=O)[O-]', 'ClCCl', 'C1CCC(C2CCCCC2)CC1',
       'CCCCOC(C)=O', 'C[N+](=O)[O-]', 'CCOC(=O)c1ccccc1', 'CCCCNCCCC',
       'CC1CO1', 'OCC1CCCO1', 'CCCCN(CCCC)CCCC', 'CCCCCCC', 'CCCC(C)CC',
       'CC1CCCCC1', 'CCC(C)C', 'CCOCC', 'CC(=O)CC(C)C', 'O=S1(=O)CCCC1',
       'O=CN1CCOCC1', 'COC(C)=O', 'c1ccsc1', 'CCCCN', 'OCCCCO',
       'CCCCOCCOCCOCCO', 'CCCOCCC', 'CC(O)COC(C)(C)C', 'CC(C)CCCC(C)CCO',
       'CCC(C)OC=O', 'CC(O)CN', 'CCC(C)OC', 'CCCC(C)CCC', 'CCC1CCCCC1',
       'CCCC(=O)CCC', 'CC(F)(F)F', 'CC(Cl)C(C)Cl', 'CC1=Cc2ccccc2C1',
       'CC1=CC=CC1', 'CC(OO)c1ccccc1', 'CC1CCc2ccccc21', 'CC1CCC(=O)O1',
       'CCC(C)(C)[NH3+]', 'CC(c1ccccc1)c1ccccc1', 'CCC(=O)CC',
       'CC1(C)CC(N=C=O)CC(C)(CN=C=O)C1', 'C=C(C)CCCCC', 'C=C(Cl)CCl',
       'C=C(C)C(=O)OC', 'C=C(C)C(C)=O', 'C=CC(=O)OCCO', 'C=CC(=O)OCC(C)O',
       'C=CBr', 'C1CS1', 'C/C=C/CCC', 'BrCBr', 'C#CCCCCCC',
       'C1=C\\CC/C=C\\CC/1', 'C1CN1', 'C/C=C\\C

In [21]:
df[(df.can_smiles_solvent == 'O') & (df.Exp_or_QM == 'Exp')]

,can_smiles_solute,can_smiles_solvent,DGsolv,index,Exp_or_QM,Train/Valid/Test,predicted,fold_number
760659,O=C(c1ccccc1)C(O)c1ccccc1,O,-11.91,20117,Exp,Train,-11.866790,9
760660,CC1CCC2(C(C)C)CC12,O,1.13,5374,Exp,Train,1.156508,9
760661,C=CC=O,O,-3.16,1749,Exp,Train,-3.120638,9
760662,COc1cc(C=O)c(Cl)c(OC)c1O,O,-7.78,13480,Exp,Train,-7.713144,9
760663,CC(O)C(F)(F)F,O,-4.16,4776,Exp,Train,-4.106543,9
...,...,...,...,...,...,...,...,...
764740,COCCCNc1nc(NC(C)C)nc(SC)n1,O,-11.87,13234,Exp,Test,-11.422320,9
764741,COc1ccccc1C(=O)O,O,-10.32,14023,Exp,Test,-9.632580,9
764742,CC(=O)CC(C)=O,O,-4.84,2414,Exp,Test,-6.094449,9
764743,COc1cccc(OC)c1O,O,-6.96,13948,Exp,Test,-7.105805,9
